# Page 1-3

In [3]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/eye-creams-moisturizers-skin-care-beauty/-/N-5viik?moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Mask",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：87 条


In [4]:
df = pd.DataFrame(product_list)
df

product_name          brand  \
0   Neutrogena Hydro Boost Under Eye Gel Cream wit...     Neutrogena   
1      RoC Derm Correxion Dual Eye Cream - 0.68 fl oz            RoC   
2   CeraVe Skin Renewing Peptide Eye Cream - 0.5 f...         CeraVe   
3   Olay Eyes Ultimate Eye Cream with Niacinamide ...           Olay   
4           e.l.f. Holy Hydration! Eye Cream - 0.53oz         e.l.f.   
..                                                ...            ...   
82  L'Oreal Paris Revitalift Triple Power Eye Crea...  L'Oreal Paris   
83             No7 Pure Retinol Eye Cream - 0.5 fl oz            No7   
84  L'Oreal Paris Revitalift Anti-Wrinkle + Firmin...  L'Oreal Paris   
85  RoC Multi Correxion Hydrate + Plump Eye Cream ...            RoC   
86  Neutrogena Rapid Wrinkle Repair Eye Cream with...     Neutrogena   

   current_price unit_price                           star_rating  \
0         $25.49        N/A  4.4 out of 5 stars with 1310 ratings   
1         $31.49        N/A  4.4 out of 5 stars with 1202 ratings   
2         $18.99        N/A  4.5 out of 5 stars with 1225 ratings   
3         $33.99        N/A  3.7 out of 5 stars with 1150 ratings   
4         $10.00        N/A  4.6 out of 5 stars with 3136 ratings   
..           ...        ...                                   ...   
82        $26.99        N/A     4 out of 5 stars with 216 ratings   
83        $14.97        N/A   4.5 out of 5 stars with 405 ratings   
84        $20.49        N/A  4.4 out of 5 stars with 1237 ratings   
85        $21.99        N/A   4.5 out of 5 stars with 157 ratings   
86        $27.99        N/A   4.3 out of 5 stars with 869 ratings   

   review_counts  is_on_sale is_free_shipping  is_3day_delivery  \
0   1310 reviews           0                0                 1   
1   1202 reviews           0                0                 1   
2   1225 reviews           0                0                 1   
3   1150 reviews           0                0                 1   
4   3136 reviews           0                0                 1   
..           ...         ...              ...               ...   
82   216 reviews           0                0                 1   
83   405 reviews           0                0                 0   
84  1237 reviews           0                0                 1   
85   157 reviews           0                0                 0   
86   869 reviews           0                0                 1   

   product_category                                       product_link  
0         Face Mask  https://www.target.com/p/neutrogena-hydro-boos...  
1         Face Mask  https://www.target.com/p/roc-derm-correxion-du...  
2         Face Mask  https://www.target.com/p/cerave-skin-renewing-...  
3         Face Mask  https://www.target.com/p/olay-eyes-ultimate-ey...  
4         Face Mask  https://www.target.com/p/e-l-f-holy-hydration-...  
..              ...                                                ...  
82        Face Mask  https://www.target.com/p/l-39-oreal-paris-revi...  
83        Face Mask  https://www.target.com/p/no7-pure-retinol-eye-...  
84        Face Mask  https://www.target.com/p/l-39-oreal-paris-revi...  
85        Face Mask  https://www.target.com/p/roc-multi-correxion-h...  
86        Face Mask  https://www.target.com/p/neutrogena-rapid-wrin...  

[87 rows x 11 columns]

In [5]:
df.to_csv("eyecream.csv",index=False)

# Page 4-6

In [6]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/eye-creams-moisturizers-skin-care-beauty/-/N-5viik?moveTo=product-list-grid&Nao=72"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Mask",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 27
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：70 条


In [7]:
df2 = pd.DataFrame(product_list2)
df2

product_name               brand  \
0   ESW Beauty Strawberry Matcha Latte Depuffing &...          ESW Beauty   
1              No7 Pure Retinol Eye Cream - 0.5 fl oz                 No7   
2   No7 Radiance+ Bright Eye Roll-on Eye Cream - 0...                 No7   
3   L'Oreal Paris Age Perfect Cell Renewal Midnigh...       L'Oreal Paris   
4   Juice Beauty Green Apple Brightening Eye Cream...        Juice Beauty   
..                                                ...                 ...   
65  Physicians Formula Rose All Day To Night Eye C...  Physicians Formula   
66            IMAGE Skincare The MAX Eye Creme 0.5 oz      IMAGE Skincare   
67   derma e Stem Cell Eye Lifting Treatment – 0.5 oz             derma e   
68  Kiehl's Creamy Eye Treatment With Avocado, 0.9...             Kiehl's   
69  Kiehl's Super Multi-Corrective Eye Zone Treatm...             Kiehl's   

   current_price unit_price                           star_rating  \
0          $5.99        N/A   4.9 out of 5 stars with 328 ratings   
1         $14.97        N/A   4.5 out of 5 stars with 405 ratings   
2         $11.99        N/A   4.2 out of 5 stars with 537 ratings   
3         $31.99        N/A   4.6 out of 5 stars with 869 ratings   
4         $45.00        N/A   3.9 out of 5 stars with 202 ratings   
..           ...        ...                                   ...   
65        $20.99        N/A    4.5 out of 5 stars with 18 ratings   
66        $67.75        N/A                                   N/A   
67        $19.96        N/A   4.4 out of 5 stars with 316 ratings   
68        $44.48        N/A  4.3 out of 5 stars with 3100 ratings   
69        $53.41        N/A   4.3 out of 5 stars with 992 ratings   

   review_counts  is_on_sale  is_free_shipping  is_3day_delivery  \
0    328 reviews           0                 0                 1   
1    405 reviews           0                 0                 0   
2    537 reviews           0                 0                 0   
3    869 reviews           0                 0                 1   
4    202 reviews           0                 1                 1   
..           ...         ...               ...               ...   
65    18 reviews           0                 0                 0   
66     0 reviews           1                 1                 0   
67   316 reviews           0                 0                 0   
68  3100 reviews           1                 1                 0   
69   992 reviews           1                 1                 0   

   product_category                                       product_link  
0         Face Mask  https://www.target.com/p/esw-beauty-strawberry...  
1         Face Mask  https://www.target.com/p/no7-pure-retinol-eye-...  
2         Face Mask  https://www.target.com/p/no7-radiance-bright-e...  
3         Face Mask  https://www.target.com/p/l-39-oreal-paris-age-...  
4         Face Mask  https://www.target.com/p/juice-beauty-green-ap...  
..              ...                                                ...  
65        Face Mask  https://www.target.com/p/physicians-formula-ro...  
66        Face Mask  https://www.target.com/p/image-skincare-the-ma...  
67        Face Mask  https://www.target.com/p/derma-e-stem-cell-eye...  
68        Face Mask  https://www.target.com/p/kiehl-s-creamy-eye-tr...  
69        Face Mask  https://www.target.com/p/kiehl-s-super-multi-c...  

[70 rows x 11 columns]

In [8]:
df2.to_csv("eyecream2.csv",index=False)


# 汇总前6页

In [9]:
df_combined_eyecream_1_6 = pd.concat([df,df2],axis=0, ignore_index=True)
df_combined_eyecream_1_6

product_name               brand  \
0    Neutrogena Hydro Boost Under Eye Gel Cream wit...          Neutrogena   
1       RoC Derm Correxion Dual Eye Cream - 0.68 fl oz                 RoC   
2    CeraVe Skin Renewing Peptide Eye Cream - 0.5 f...              CeraVe   
3    Olay Eyes Ultimate Eye Cream with Niacinamide ...                Olay   
4            e.l.f. Holy Hydration! Eye Cream - 0.53oz              e.l.f.   
..                                                 ...                 ...   
152  Physicians Formula Rose All Day To Night Eye C...  Physicians Formula   
153            IMAGE Skincare The MAX Eye Creme 0.5 oz      IMAGE Skincare   
154   derma e Stem Cell Eye Lifting Treatment – 0.5 oz             derma e   
155  Kiehl's Creamy Eye Treatment With Avocado, 0.9...             Kiehl's   
156  Kiehl's Super Multi-Corrective Eye Zone Treatm...             Kiehl's   

    current_price unit_price                           star_rating  \
0          $25.49        N/A  4.4 out of 5 stars with 1310 ratings   
1          $31.49        N/A  4.4 out of 5 stars with 1202 ratings   
2          $18.99        N/A  4.5 out of 5 stars with 1225 ratings   
3          $33.99        N/A  3.7 out of 5 stars with 1150 ratings   
4          $10.00        N/A  4.6 out of 5 stars with 3136 ratings   
..            ...        ...                                   ...   
152        $20.99        N/A    4.5 out of 5 stars with 18 ratings   
153        $67.75        N/A                                   N/A   
154        $19.96        N/A   4.4 out of 5 stars with 316 ratings   
155        $44.48        N/A  4.3 out of 5 stars with 3100 ratings   
156        $53.41        N/A   4.3 out of 5 stars with 992 ratings   

    review_counts  is_on_sale is_free_shipping  is_3day_delivery  \
0    1310 reviews           0                0                 1   
1    1202 reviews           0                0                 1   
2    1225 reviews           0                0                 1   
3    1150 reviews           0                0                 1   
4    3136 reviews           0                0                 1   
..            ...         ...              ...               ...   
152    18 reviews           0                0                 0   
153     0 reviews           1                1                 0   
154   316 reviews           0                0                 0   
155  3100 reviews           1                1                 0   
156   992 reviews           1                1                 0   

    product_category                                       product_link  
0          Face Mask  https://www.target.com/p/neutrogena-hydro-boos...  
1          Face Mask  https://www.target.com/p/roc-derm-correxion-du...  
2          Face Mask  https://www.target.com/p/cerave-skin-renewing-...  
3          Face Mask  https://www.target.com/p/olay-eyes-ultimate-ey...  
4          Face Mask  https://www.target.com/p/e-l-f-holy-hydration-...  
..               ...                                                ...  
152        Face Mask  https://www.target.com/p/physicians-formula-ro...  
153        Face Mask  https://www.target.com/p/image-skincare-the-ma...  
154        Face Mask  https://www.target.com/p/derma-e-stem-cell-eye...  
155        Face Mask  https://www.target.com/p/kiehl-s-creamy-eye-tr...  
156        Face Mask  https://www.target.com/p/kiehl-s-super-multi-c...  

[157 rows x 11 columns]

In [10]:
df_combined_eyecream_1_6.to_csv("eyecream_1_6.csv",index=False)

# Page 7-9

In [1]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]


random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/eye-creams-moisturizers-skin-care-beauty/-/N-5viik?moveTo=product-list-grid&Nao=144"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list3 = []

def scrape_products():
    global product_list3, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list3.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Eyecream",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 4
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list3)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 4
❌ 没有找到下一页按钮，结束爬取
✅ WebDriver 关闭
📦 共抓取商品（去重后）：52 条


In [2]:
df3 = pd.DataFrame(product_list3)
df3

product_name                brand  \
0   Unique Bargains Eye Cream Massage Sticks Beaut...      Unique Bargains   
1   Dr. PawPaw Your Gorgeous Skin Brightening Eye ...           Dr. PAWPAW   
2   L'Oreal Paris Revitalift Derm Intensives Hyalu...        L'Oreal Paris   
3   No7 Future Renew Damage Reversal Eye Serum - 0...                  No7   
4   BYOMA De-Puff and Brightening Eye Gel - 0.68 f...                BYOMA   
5   Good Molecules Yerba Mate Wake Up Eye Gel - 0....       Good Molecules   
6   Versed Weekend Glow Brightening Eye Gel - 0.5 ...               Versed   
7      cocokind Peptide Fine Line Eye Gel - 0.5 fl oz             cocokind   
8   Beekman 1802 Dewy Eyed Illuminating Depuffing ...         Beekman 1802   
9   Clinique For Men Anti-Fatigue Eye Gel - 0.5 fl...             Clinique   
10        Weleda Sheer Hydration Eye Gel - 0.34 fl oz               Weleda   
11                TruSkin Peptide Eye Gel - 0.5 fl oz              TruSkin   
12  PETER THOMAS ROTH Water Drench Hyaluronic Clou...    PETER THOMAS ROTH   
13  ESW Beauty Strawberry Matcha Latte Depuffing &...           ESW Beauty   
14  Cosmedica Skincare Cooling Eye Edit Ice Globe ...   Cosmedica Skincare   
15   Acure Resilience Eye Balm for Fine Lines - 0.5oz                Acure   
16  24k Gold Under Eye Patches for Puffy Eyes and ...           Blue Oasis   
17  Babe Lash Original Essential Eyelash Serum (3 ...        BABE ORIGINAL   
18  Day & Night Under Eye Patches for Puffy Eyes &...               Risque   
19                        Baebody Eye Gel - 1.7 fl oz              Baebody   
20                      Garnier Skin Renew Eye Roller              Garnier   
21                        EltaMD Renew Eye Gel 0.5 oz               EltaMD   
22  Vibriance Eye Renewal Serum - Eye Serum for Da...            Vibriance   
23              Stye Eye Therapy Warming Compress 1Ct                 Stye   
24  AminoGenesis DermaScyne Gone in Sixty Seconds ...         AminoGenesis   
25  Babe Lash Original Essential Eyelash Serum (6 ...        BABE ORIGINAL   
26                  Neova Super Eye Lift Serum 0.5 oz                NEOVA   
27         Lumene Lahde Hydrating Eye Gel - 0.5 fl oz               Lumene   
28      Eminence Neroli Age Corrective Eye Serum 1 oz             Eminence   
29   Mei Apothecary Cloudberry Dew Eye Gel Mask - 3ct       Mei Apothecary   
30  Lumene Nordic-C Valo Glow Boost Eye Serum - 1 ...               Lumene   
31  StriVectin Super-C Dark Circle Brightening Eye...           StriVectin   
32            Kiehl's Age Defender Eye Repair, 0.5 oz              Kiehl's   
33    L'Occitane Immortelle Precious Eye Balm 0.52 oz           L'Occitane   
34      Pursonic Professional Anti Aging Eye Gel 2 Oz             Pursonic   
35              Guerlain Super Aqua Eye Serum, 0.5 oz             Guerlain   
36  L'Occitane Immortelle Overnight Reset Eye Seru...           L'Occitane   
37            Beauty by Earth Hyaluronic Acid Eye Gel      Beauty by Earth   
38  Elizabeth Arden Advanced Ceramide Capsules Dai...      Elizabeth Arden   
39  Radiance Boost Bundle: Derma Roller, Vitamin C...             Pursonic   
40                Neova Illuminating Eye Serum 0.5 oz                NEOVA   
41                 EltaMD Renew Eye Gel 0.5 oz 2 Pack               EltaMD   
42   Retinol By Robanda Active Renewal Eye Gel, 0.5oz   Retinol By Robanda   
43              Pursonic Wrinkle Care Eye Gel Patches             Pursonic   
44  Pursonic All Natural Professional Anti Aging E...             Pursonic   
45   Pursonic Ultimate Skincare Ampoule & Eye Gel Set             Pursonic   
46  Pursonic Complete Eye Care and Skin Refresh Bu...             Pursonic   
47  Eye Contour Fluid by LErbolario for Men - 0.5 ...           LErbolario   
48    Versed Water Rush Hydrating Eye Gel - 0.5 fl oz               Versed   
49  Rule Cosmeceuticals Stem Cell Eye Serum with B...  Rule Cosmeceuticals   
50     Optimel Manuka+™ Forte Eye Gel, 0.35 oz (10 g)          

In [3]:
df3.to_csv("eyecream_7_9.csv",index=False)